In [ ]:
from multiversx_sdk_wallet import Mnemonic
from multiversx_sdk_core import Address
from multiversx_sdk_core import AddressFactory

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)

In [ ]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print(secret_key.hex())
print("Public key (hex-encoded):", secret_key.hex())

In [ ]:
factory = AddressFactory("erd")
address = factory.create_from_hex("19b9cf83c67a1c21ed313ac4d75e66df71d267a15dcf609f37ca5695ee86bb7f")

In [ ]:
import os
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [ ]:
result = supabase.table('chatbot_messages') \
  .insert({
    "user_id": "123456789",
    "chat_id": "chat_001",
    "text": "Hello, how can I help you?",
    "is_bot": True
  }) \
  .execute()
print(result)

## Code Generation

In [ ]:
# from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
load_dotenv()

In [ ]:
# Clone
repo_path = "./github-scrapper/code_scrapped"
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".rs"],
    parser=LanguageParser(language=Language.RUST, parser_threshold=500)
)
documents = loader.load()
len(documents)
rust_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.RUST, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = rust_splitter.split_documents(documents)
len(texts)

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

llm = ChatOpenAI(model_name="gpt-4") 
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

question = "Generate multiversx contract code: Create a contract that holds the number 18 and provides a function to increase it."
result = qa(question)
result['answer']

In [ ]:
print(result['answer'])

In [ ]:
rust_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.RUST, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = rust_splitter.split_documents(documents)
len(texts)

In [ ]:

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
llm = ChatOpenAI(model_name="gpt-4") 
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "Generate multiversx contract code: Create a contract that holds the number 18 and provides a function to increase it."
result = qa(question)
result['answer']

In [ ]:
print(result['answer'])

In [ ]:
print(result['answer'])

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

base_url = "https://docs.multiversx.com/"

def get_links_from_page(soup, base_url):
    # Extract all links on the current page that are relative and prepend with base URL
    links = [base_url + a['href'] for a in soup.find_all('a', href=True) 
             if a['href'].startswith('/') and 'multiversx' in base_url.lower()]
    return links


url = "https://docs.multiversx.com/"
all_contents = []
visited_urls = set()
to_visit_urls = [url]

while to_visit_urls:
    current_url = to_visit_urls.pop()
    if current_url in visited_urls:
        continue

    response = requests.get(current_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Store content
    all_contents.append(soup.prettify())

    # Get all links from the current page
    links = get_links_from_page(soup, base_url)
    to_visit_urls.extend(links)
    print(to_visit_urls)
    visited_urls.add(current_url)

print(links)


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

def get_links_from_page(soup, base_url):
    # Extract all relative links on the current page and convert to absolute URL
    links = [urljoin(base_url, a['href']) for a in soup.find_all('a', href=True)
             if a['href'].startswith('/')]
    return links

def recursive_scrape(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return visited_urls

    response = requests.get(url)
    if response.status_code != 200:
        return visited_urls  # If we don't get a successful response, return visited URLs

    soup = BeautifulSoup(response.content, 'html.parser')
    visited_urls.add(url)

    # Get all relative links from the current page
    links = get_links_from_page(soup, url)
    print(visited_urls)
    for link in links:
        if link not in visited_urls:
            visited_urls.update(recursive_scrape(link, visited_urls))
    return visited_urls

start_url = "https://docs.multiversx.com/"
all_visited_links = recursive_scrape(start_url)
print(all_visited_links)

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_links_from_page(soup, base_url):
    # Extract all relative links on the current page and convert to absolute URL
    links = [urljoin(base_url, a['href']) for a in soup.find_all('a', href=True)
             if a['href'].startswith('/')]
    return links

def get_text_from_page(soup):
    # Get textual content from the page
    return " ".join([t.get_text() for t in soup.find_all(True) if t.name not in ['script', 'style']])

def recursive_scrape(url, visited_urls=None, content_list=None):
    if visited_urls is None:
        visited_urls = set()
    if content_list is None:
        content_list = []

    if url in visited_urls:
        return content_list

    response = requests.get(url)
    if response.status_code != 200:
        return content_list  # If we don't get a successful response, return the collected content so far

    soup = BeautifulSoup(response.content, 'html.parser')
    visited_urls.add(url)

    # Get the textual content of the current page
    page_content = get_text_from_page(soup)
    content_list.append(page_content)

    # Get all relative links from the current page
    links = get_links_from_page(soup, url)

    print(visited_urls)
    for link in links:
        if link not in visited_urls:
            content_list = recursive_scrape(link, visited_urls, content_list)

    return content_list

start_url = "https://docs.multiversx.com/"
all_content = recursive_scrape(start_url)

# Printing all the content collected
for content in all_content:
    print(content)


filename = "scraped_content.txt"

# Write the collected content to the file
with open(filename, 'w', encoding='utf-8') as f:
    for content in all_content:
        f.write(content + "\n\n")
        print(content)  # If you still want to print the content to the console

print(f"Content has been saved to {filename}")


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DeepLake
from dotenv import load_dotenv
load_dotenv()

loader = TextLoader("./scraped_v2.txt")
documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,
)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [ ]:
username = "commanderastern"
dataset_path = f"hub://{username}/qanda_mx2"
db = DeepLake(
    dataset_path=dataset_path, embedding_function =embeddings, overwrite=True
)
db.add_documents(docs)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat

db = DeepLake(
    dataset_path='./qanda_mx2/', embedding_function =embeddings, read_only=True
)
qa = RetrievalQA.from_chain_type(
    llm=OpenAIChat(model="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=db.as_retriever(),
)

In [ ]:
query = "With what macro is a module trait defined"
qa.run(query)